In [ ]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# For this exercise you will build a cats v dogs classifier
# using the Cats v Dogs dataset from TFDS.
# Be sure to use the final layer as shown 
#     (Dense, 2 neurons, softmax activation)
#
# The testing infrastructre will resize all images to 224x224 
# with 3 bytes of color depth. Make sure your input layer trains
# images to that specification, or the tests will fail.
#
# Make sure your output layer is exactly as specified here, or the 
# tests will fail.

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

dataset_name = 'cats_vs_dogs'
# dataset = tfds.load(name=dataset_name, split=tfds.Split.TRAIN)
dataset_t = tfds.load(name=dataset_name, split='train[:80%]')
dataset_v = tfds.load(name=dataset_name, split='train[80%:]')

def preprocess(data):
    # YOUR CODE HERE
    x = data['image']
    y = data['label']
    x = tf.cast(x,tf.float32) / 255.0
    x = tf.image.resize(x, size=(224,224))
    return x, y

def solution_model():
    train_dataset = dataset_t.map(preprocess).batch(32)
    valid_dataset = dataset_v.map(preprocess).batch(32)
    
    conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
    conv_base.trainable = False
    
    model = Sequential([
        conv_base, 
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        # YOUR CODE HERE, BUT MAKE SURE YOUR LAST LAYER HAS 2 NEURONS ACTIVATED BY SOFTMAX
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', 
                 metrics=['acc'])
    checkpoint_path = "my_checkpoint.ckpt"
    checkpoint = ModelCheckpoint(filepath= checkpoint_path, save_weights_only=True,
                                save_best_only=True, monitor='val_loss', verbose=1)
    model.fit(train_dataset, validation_data=(valid_dataset), epochs=20
             ,callbacks=[checkpoint])
    
    model.load_weights(checkpoint_path)
    
    return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("cat3_catsdogs2_vgg.h5")


Epoch 1/20
 29/582 [>.............................] - ETA: 15:28 - loss: 2.8178 - acc: 0.5733